In [1]:
## FMP is being currently used to source historical price data - it is exporting historical data to individual CSV files for each symbol 
## In theory - should be possible to parse the CSV files for historical data, and append missing data from FMP via IEX and make more 
## complete data frames. FMP has an issue where it can't source fundamentals data (market cap, outstandingshares) for ETFs. IEX can. 
## Need to come up with code to fill in the gaps from FMP with IEX data. Unlike FMP, IEX has limited API calls, so should attempt to 
## source FMP first and then call IEX to fill in gaps 

In [2]:
import pandas as pd

import glob

from pathlib import Path
import csv

import os
import requests
import json

#import quandl

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
## Constants 

## Set start date variable - dataframes will be created starting from this date
start_date = '2020-01-01'
end_date = '2021-10-29'
default_date_range = '2y' ## Default Range for IEX functions - don't need more at the moment

In [4]:
## IEX Constants
iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

## Redundant Assignment but improves Readability throughout code 
real_token = iex_api_key
test_token = iex_test_api_key

base_url_iex = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

## IEX Status Test 
test_resp = requests.get(base_url_iex + 'status')
test_resp

<Response [200]>

In [5]:
token_status = test_token ## Set to either real token or test token for IEX

In [6]:
## FMP Constants 
fmpbase_urlv3 = 'https://fmpcloud.io/api/v3/'
fmpbase_urlv4 = 'https://fmpcloud.io/api/v4/'
api_key = os.getenv("FMP_CLOUD_API_KEY")

## FMP Functions - this file should not need to call the historical data function. 
##                 This file should import the CSV files with historical data and then append IEX testing data
##                 to any data FMP cannot source. 
def get_FMP_historical_data(symbol, startDate=start_date, endDate=end_date, apiKey=api_key):
    url_hist_price = fmpbase_urlv3+'historical-price-full/'
    url_hist_query_with_date = url_hist_price+symbol+'?from='+startDate+'&to='+endDate+'&apikey='+apiKey
    resp_data = requests.get(url_hist_query_with_date)
    json_ = resp_data.json()
    data = json_['historical']
    df = pd.DataFrame(data)
    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.reindex(index=df.index[::-1]) ## Reverse the DataFrame 
    df.set_index('Date',inplace=True)
    df.drop(columns='label',inplace=True)
    return df

def get_float_data_FMP(symbol):
    url_float_shares = fmpbase_urlv4+'shares_float?symbol='
    url_query_float_data = url_float_shares+symbol+'&apikey='+api_key
    resp_data = requests.get(url_query_float_data)
    #df = pd.DataFrame(resp_data.json())
    json_ = resp_data.json()
    return json_[0]

def get_company_profile_FMP_json(symbol):
    ## https://fmpcloud.io/api/v3/profile/AAPL?apikey='yourkeyhere'
    url_company_profile_url = fmpbase_urlv3+'profile/'+symbol+'?apikey='+api_key
    resp_data = requests.get(url_company_profile_url)
    json_response = resp_data.json()
    return json_response[0]

# def save_and_export_raw_df_csv(data, symbol):
#     path = ('../FilesExportIndividualStockDFs/'+symbol+'_combined_df.csv')
#     data.to_csv(path)

In [7]:
# # ## Import FTD File using CSV 
# # ftd_df = pd.read_csv(
# #     Path('../Resources/ftd_all_data.csv'),
# #     index_col=0, parse_dates=True
# # )

# ## Don't need FTD file at the moment. Import symbol_df as a way to get symbols to iterate through saved CSVs. 
# ## Some symbols will fail, need to create function that skips missed indexes 

# ## Import Symbol and CUSIP list using CSV
# symbol_df = pd.read_csv(
#     Path('../Resources/symbol_all_list.csv'),
#     index_col=0
# )

In [8]:
## Test importing PKL files to get entire database dict 

## Use pickle module to import and export and save files
import pickle
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [31]:
# ## How to find index value of specific stock 
# index_value = 0
# for i in complete_key_list:
#     if i == 'ACP':
#         print(index_value)
#         break
#     index_value +=1

42


In [9]:
x = 0
y = 49
increment = 50
complete_data_dict = {}
## Current last file is 8450 
while x < 8450:
    str_symbol1 = str(x)
    str_symbol2 = str(y)
    
    pkl_path = Path('../FilesExportCompleteFMP/data_complete_'+str_symbol1+'_'+str_symbol2+'.pkl')
    data_obj = load_obj(pkl_path)
    data_obj_key_list = []
    for key in data_obj.keys():
        data_obj_key_list.append(key)  
    for i in data_obj_key_list:
        data = data_obj[i]
        complete_data_dict[i] = data
        
    x += increment
    y += increment
    
complete_key_list = []
for key in complete_data_dict.keys():
    complete_key_list.append(key)

In [10]:
len(complete_data_dict)

1373

In [11]:
complete_data_dict['GME']['companyProfile']['isEtf']

False

In [34]:
etf_data_dict = {} 
equity_data_dict = {}
no_profile_data_dict = {}
error_count = 0                  ## Can use to check index in complete_key_list when loop fails 
for i in complete_key_list:
    #symbol = i 
    if complete_data_dict[i]['companyProfile'] != 0:
        if complete_data_dict[i]['companyProfile']['isEtf'] == True:   ## Add to etf_data_dict
            data = complete_data_dict[i]
            etf_data_dict[i] = data
        else:                                   ## Add to equity_data_dict
            data = complete_data_dict[i]
            equity_data_dict[i] = data
    else:                                      ## Add to no_profile_data_dict
        data = complete_data_dict[i]
        no_profile_data_dict[i] = data
    error_count += 1 
        
etf_key_list = []
for key in etf_data_dict.keys():
    etf_key_list.append(key)
len(etf_data_dict)

147

In [38]:
error_count # 42, 'ACP' gave error - no companyProfile data 
## if error_count == len(complete_data_dict) then was successful

1373

In [35]:
value = complete_key_list[42]
value
complete_data_dict[value]['companyProfile']

#complete_data_dict['GME']['companyProfile']['isEtf']

0

In [44]:
## Test with 'ARKQ'
etf_data_dict['ARKQ']['Fundamentals']

{'sharesOutstanding': 0,
 'floatShares': 0,
 'marketCap': 184056034,
 'debt_ratio': 'debt_ratioValue',
 'price_earnings': 'price_earningsValue',
 'exchange': 'New York Stock Exchange Arca',
 'final_close_price': 84.800003}

In [49]:
## Load IEX to get ETF statistics 
def get_IEX_statistics(stock_ticker, token=token_status):
    if token == test_token:
        resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/stats/?token='+test_token)
        data_json = resp_data.json()
    elif token == real_token:
        resp_data = requests.get(base_url_iex+'stock/'+stock_ticker+'/stats/?token='+real_token)
        data_json = resp_data.json()
        
    return data_json

In [46]:
## Test changing value in current dict 
etf_data_dict['ARKQ']['Fundamentals']['sharesOutstanding'] = 10000

In [58]:
etf_data_dict['ARKQ']

{'Fundamentals': {'sharesOutstanding': 10000,
  'floatShares': 0,
  'marketCap': 184056034,
  'debt_ratio': 'debt_ratioValue',
  'price_earnings': 'price_earningsValue',
  'exchange': 'New York Stock Exchange Arca',
  'final_close_price': 84.800003},
 'dataFrame':                  open       high        low      close   adjClose    volume  \
 Date                                                                          
 2020-01-02  37.580002  37.891998  37.516998  37.891998  37.564457   18100.0   
 2020-01-03  37.570000  37.990002  37.570000  37.797001  37.470280   28300.0   
 2020-01-06  37.639999  37.859001  37.435001  37.842999  37.515881   27400.0   
 2020-01-07  38.009998  38.375000  37.860001  38.310001  37.978848   23800.0   
 2020-01-08  38.410000  39.000000  38.330002  38.869999  38.534004   43900.0   
 ...               ...        ...        ...        ...        ...       ...   
 2021-10-25  82.490000  84.755000  82.210000  84.390000  84.390000  330479.0   
 2021-10-26  84.

In [55]:
## Test using IEX to get data 
token_status = test_token
symbol = 'arkq'
etf_stats = get_IEX_statistics(symbol)

In [57]:
etf_stats['sharesOutstanding']

31436616

In [53]:
## Create test df with ETFs and append with test data from IEX 
#etf_data_dict 
test_etf_key_list = etf_key_list[0:50]
len(test_etf_key_list)

50

In [59]:
test_etf_data_dict = {}
for i in test_etf_key_list:
    data = etf_data_dict[i]
    test_etf_data_dict[i] = data

In [60]:
for i in test_etf_key_list:
    symbol = i 
    iex_etf_stats = get_IEX_statistics(symbol)
    sharesOutstanding = iex_etf_stats['sharesOutstanding']   
    test_etf_data_dict[i]['Fundamentals']['sharesOutstanding'] = sharesOutstanding
    test_etf_data_dict[i]['iex_statistics'] = iex_etf_stats

In [62]:
## Success. Works with test token and appends data. 
# for i in test_etf_key_list:
#     sharesOutstanding = test_etf_data_dict[i]['Fundamentals']['sharesOutstanding']
#     print(sharesOutstanding)

In [65]:
#arkq_real_stats = get_IEX_statistics('ARKQ',token=real_token)

In [75]:
arkq_real_stats

{'companyName': 'ARK Investment Management LLC',
 'marketcap': 2567888000,
 'week52high': 101.11,
 'week52low': 67.44,
 'week52highSplitAdjustOnly': 101.11,
 'week52lowSplitAdjustOnly': 67.44,
 'week52change': 0.2254461047439429,
 'sharesOutstanding': 30400000,
 'float': 0,
 'avg10Volume': 388788,
 'avg30Volume': 373532,
 'day200MovingAvg': 82.05,
 'day50MovingAvg': 84.42,
 'employees': 0,
 'ttmEPS': 0,
 'ttmDividendRate': 0.6560999751091003,
 'dividendYield': 0.007767254351948625,
 'nextDividendDate': '',
 'exDividendDate': '2020-12-29',
 'nextEarningsDate': '',
 'peRatio': 0,
 'beta': 0,
 'maxChangePercent': 3.3864340944378952,
 'year5ChangePercent': 2.898195578937653,
 'year2ChangePercent': 1.4684395090590296,
 'year1ChangePercent': 0.2338591878469181,
 'ytdChangePercent': 0.10722244068685272,
 'month6ChangePercent': 0.06305059149257497,
 'month3ChangePercent': 0.02549471895107436,
 'month1ChangePercent': 0.030373261771163618,
 'day30ChangePercent': 0.0009479796184381772,
 'day5Chan

In [72]:
test_etf_data_dict['ARKQ']['Fundamentals']

{'sharesOutstanding': 30516286,
 'floatShares': 0,
 'marketCap': 184056034,
 'debt_ratio': 'debt_ratioValue',
 'price_earnings': 'price_earningsValue',
 'exchange': 'New York Stock Exchange Arca',
 'final_close_price': 84.800003}

In [73]:
FMP_arkq_data = get_company_profile_FMP_json('ARKQ')

In [74]:
FMP_arkq_data

{'symbol': 'ARKQ',
 'price': 84.47,
 'beta': 0.0,
 'volAvg': 300246,
 'mktCap': 184056034,
 'lastDiv': 0.656,
 'range': '67.44-101.11',
 'changes': 1.5,
 'companyName': 'ARK Autonomous Technology & Robotics ETF',
 'currency': 'USD',
 'cik': None,
 'isin': 'US00214Q2030',
 'cusip': None,
 'exchange': 'New York Stock Exchange Arca',
 'exchangeShortName': 'AMEX',
 'industry': '',
 'website': '',
 'description': 'The investment seeks long-term growth of capital.\n The fund is an actively-managed exchange-traded fund that will invest under normal circumstances primarily (at least 80% of its assets) in domestic and foreign equity securities of autonomous technology and robotics companies that are relevant to the fund&#39;s investment theme of disruptive innovation. Substantially all of the fund&#39;s assets will be invested in equity securities, including common stocks, partnership interests, business trust shares and other equity investments or ownership interests in business enterprises. T